### Задание 1.
Задание 1. 
Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы, сделать выводы  


In [1]:
!pip install pyconll


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install -U spacy
!python -m spacy info
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 4.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 kB 4.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 4.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 4.9 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are 


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [3]:
import pyconll

import pandas as pd

import nltk
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, classification_report


In [5]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train-a.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [6]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [7]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [8]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [9]:
fdata_train[:3]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')],
 [('В', 'ADP'),
  ('приемной', 'NOUN'),
  ('его', 'PRON'),
  ('с', 'ADP'),
  ('утра', 'NOUN'),
  ('ожидали', 'VERB'),
  ('посетители', 'NOUN'),
  (',', 'PUNCT'),
  ('-', 'PUNCT'),
  ('кое-кто', 'PRON'),
  ('с', 'ADP'),
  ('важными', 'ADJ'),

In [10]:
unigram_tagger = UnigramTagger(fdata_train)
acc_u = unigram_tagger.evaluate(fdata_test)
display(acc_u)

/var/folders/wj/hqlvffyx73l63jfv3pbxljjr0000gn/T/ipykernel_10291/4208555745.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_u = unigram_tagger.evaluate(fdata_test)


0.823732013802982

In [11]:
bigram_tagger = BigramTagger(fdata_train)
acc_b = bigram_tagger.evaluate(fdata_test)
display(acc_b)

/var/folders/wj/hqlvffyx73l63jfv3pbxljjr0000gn/T/ipykernel_10291/2907996023.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_b = bigram_tagger.evaluate(fdata_test)


0.6093886320724006

In [12]:
trigram_tagger = TrigramTagger(fdata_train)
acc_t = trigram_tagger.evaluate(fdata_test)
display(acc_t)

/var/folders/wj/hqlvffyx73l63jfv3pbxljjr0000gn/T/ipykernel_10291/678438604.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_t = trigram_tagger.evaluate(fdata_test)


0.1778631421316492

In [13]:
combination_1 = [BigramTagger, TrigramTagger]
combination_2 = [TrigramTagger, UnigramTagger]
combination_3 = [UnigramTagger, BigramTagger]
combination_4 = [BigramTagger, TrigramTagger, UnigramTagger]

In [14]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

acc_bo = []

for combination in [combination_1, combination_2, combination_3, combination_4]:
    backoff = nltk.DefaultTagger('NN') 
    tag = backoff_tagger(fdata_train,  
                         combination,  
                         backoff = backoff) 

    acc_bo.append(tag.evaluate(fdata_test))

acc_bo

/var/folders/wj/hqlvffyx73l63jfv3pbxljjr0000gn/T/ipykernel_10291/2746214562.py:14: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_bo.append(tag.evaluate(fdata_test))


[0.1750309264926102,
 0.21248779217396965,
 0.8275343446838986,
 0.3623152548994075]

In [ ]:
result = pd.DataFrame({'Tagger': ['UnigramTagger', 'BigramTagger', 'TrigramTagger', 'BigramTagger & TrigramTagger', 'TrigramTagger & UnigramTagger', 'UnigramTagger & BigramTagger', 'All taggers'], 'Accuracy' : [acc_u, acc_b, acc_t] + acc_bo})
result.sort_values('Accuracy', ascending=False)

,Tagger,Accuracy
5,UnigramTagger & BigramTagger,0.827534
0,UnigramTagger,0.823732
1,BigramTagger,0.609389
6,All taggers,0.362315
4,TrigramTagger & UnigramTagger,0.212488
2,TrigramTagger,0.177863
3,BigramTagger & TrigramTagger,0.175031


In [15]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [16]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [17]:
vectorizers = [CountVectorizer(ngram_range=(1, 3), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000, 10000]
hvectorizer = [HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=feat) for feat in n_features]
hvectorizer_word = [HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=feat) for feat in n_features]

In [18]:
f1_scores = []
acc_scores = []

for vectorizer in vectorizers + vectorizers_word + hvectorizer + hvectorizer_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok[:115000])
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels[:115000], pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels[:115000], pred)
    acc_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels[:115000], pred, target_names=le.classes_))

/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.92      0.91      0.92     11247
         ADP       0.98      1.00      0.99     10255
         ADV       0.92      0.90      0.91      5986
         AUX       0.81      0.97      0.88      1058
       CCONJ       0.88      0.98      0.93      4276
         DET       0.88      0.75      0.81      2978
        INTJ       0.33      0.36      0.35        11
        NOUN       0.92      0.95      0.94     27241
      NO_TAG       1.00      1.00      1.00       197
         NUM       0.86      0.90      0.88      1436
        PART       0.95      0.78      0.86      3762
        PRON       0.83      0.89      0.86      5346
       PROPN       0.79      0.59      0.67      4315
       PUNCT       1.00      1.00      1.00     21941
       SCONJ       0.81      0.91      0.86      2176
         SYM       1.00      0.68      0.81        53
        VERB       0.94     

/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TfidfVectorizer(analyzer='char', ngram_range=(1, 3))


/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

...

/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

         ADJ       0.63      0.33      0.44     11247
         ADP       0.91      0.48      0.63     10255
         ADV       0.82      0.72      0.77      5986
         AUX       0.79      0.95      0.86      1058
       CCONJ       0.92      0.19      0.31      4276
         DET       0.69      0.70      0.70      2978
        INTJ       0.00      0.00      0.00        11
        NOUN       0.70      0.61      0.65     27241
      NO_TAG       0.00      0.00      0.00       197
         NUM       0.66      0.48      0.55      1436
        PART       0.91      0.76      0.83      3762
        PRON       0.79      0.79      0.79      5346
       PROPN       0.54      0.13      0.21      4315
       PUNCT       0.44      1.00      0.62     21941
       SCONJ       0.76      0.88      0.82      2176
         SYM       0.00      0.00      0.00        53
        VERB       0.67      0.36      0.47     12617
           X       0.00    

/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


HashingVectorizer(n_features=10000, ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.73      0.36      0.48     11247
         ADP       0.95      0.48      0.64     10255
         ADV       0.88      0.75      0.81      5986
         AUX       0.81      0.88      0.84      1058
       CCONJ       0.94      0.19      0.31      4276
         DET       0.72      0.75      0.74      2978
        INTJ       0.00      0.00      0.00        11
        NOUN       0.77      0.64      0.70     27241
      NO_TAG       0.00      0.00      0.00       197
         NUM       0.73      0.57      0.64      1436
        PART       0.94      0.77      0.84      3762
        PRON       0.83      0.78      0.81      5346
       PROPN       0.67      0.15      0.24      4315
       PUNCT       0.42      1.00      0.59     21941
       SCONJ       0.78      0.88      0.83      2176
         SYM       0.00      0.00      0.00        53
        VERB       0.76  

/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + hvectorizer + hvectorizer_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927835
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.921194
9,"HashingVectorizer(analyzer='char', n_features=...",0.903654
8,"HashingVectorizer(analyzer='char', n_features=...",0.901786
7,"HashingVectorizer(analyzer='char', n_features=...",0.897062
6,"HashingVectorizer(analyzer='char', n_features=...",0.895273
2,"HashingVectorizer(analyzer='char', n_features=...",0.882224
4,"TfidfVectorizer(ngram_range=(1, 3))",0.664483
3,"CountVectorizer(ngram_range=(1, 3))",0.652817
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.617468


In [20]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + hvectorizer + hvectorizer_word,
                            'Accuracy': acc_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929513
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.923617
9,"HashingVectorizer(analyzer='char', n_features=...",0.907417
8,"HashingVectorizer(analyzer='char', n_features=...",0.905487
7,"HashingVectorizer(analyzer='char', n_features=...",0.900783
6,"HashingVectorizer(analyzer='char', n_features=...",0.898939
2,"HashingVectorizer(analyzer='char', n_features=...",0.885165
4,"TfidfVectorizer(ngram_range=(1, 3))",0.654643
3,"CountVectorizer(ngram_range=(1, 3))",0.643870
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.627426


Судя по результатам комбинации таггеров лидирует по Accuracy смесь из UnigramTagger & BigramTagger с результатом - 0.93
Во втором и третьем тестах лидирует CountVectorizer с f-score 0.927826 и accuracy 0.929504

### Задание 2. 
Проверить, насколько хорошо работает NER
Данные брать из http://www.labinform.ru/pub/named_entities/
проверить NER из nltk/spacy/deeppavlov.
написать свой NER, попробовать разные подходы.
передаём в сетку токен и его соседей.
передаём в сетку только токен.
свой вариант.
сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [21]:
!pip install -U spacy
!python -m spacy info
!python -m spacy download ru_core_news_sm


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip

============================== Info about spaCy ==============================

spaCy version    3.5.2                         
Location         /Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/spacy
Platform         macOS-10.15.7-x86_64-i386-64bit
Python version   3.9.7                         
Pipelines        ru_core_news_sm (3.5.0)       

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 3.5 MB/s eta 0:00:0000:0100:01



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [22]:
!pip install navec


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [23]:
!pip install slovnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [24]:
!pip install ipymarkup


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [25]:
!pip install sklearn_crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.6/183.6 kB 2.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [26]:
!pip install patool

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 889.4 kB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [27]:
import os
import glob

import csv
from razdel import tokenize

from spacy import displacy
import ru_core_news_sm

from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup

import sklearn_crfsuite
from sklearn_crfsuite import metrics

In [28]:
import patoolib
patoolib.extract_archive("collection3.zip", outdir="collection3")

patool: Extracting collection3.zip ...
patool: ... collection3.zip extracted to `collection3'.


'collection3'

In [29]:
fileDir = r"collection3"
fileExt = r".txt"
documents_txt = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
text_list = []
for file in documents_txt:
    doc = open('collection3/' + file, encoding='utf-8')
    text = doc.read()
    text_list.append(text)
    
data_text = pd.DataFrame({'text': text_list })
data_text

,text


In [30]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/evgeniilukyanov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/evgeniilukyanov/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [32]:
document = data_text.text[0]
nltk.pos_tag(nltk.word_tokenize(document))

KeyError: 0

In [44]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [33]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

NameError: name 'document' is not defined

In [47]:
pd.read_csv('collection3/001.ann', delimiter='\t')

,T1,LOC 0 6,Россия
0,T2,LOC 50 53,США
1,T3,LOC 57 63,Грузию
2,T4,LOC 87 93,МОСКВА
3,T5,ORG 103 114,РИА Новости
4,T6,LOC 116 122,Россия
5,T7,LOC 141 144,США
6,T8,LOC 161 168,Тбилиси
7,T9,LOC 301 307,России
8,T10,PER 308 324,Григорий Карасин
9,T11,LOC 383 386,США


In [34]:
nlp = ru_core_news_sm.load()
ny_bb = data_text.text[0]
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

KeyError: 0

In [35]:
for token in article:
    print(token.text, token.pos_, token.dep_)

NameError: name 'article' is not defined

In [36]:
pd.read_csv('collection3/001.ann', delimiter='\t', header=None)

FileNotFoundError: [Errno 2] No such file or directory: 'collection3/001.ann'

In [72]:
text = data_text.text[0]
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)

markup = ner(text)
show_markup(markup.text, markup.spans)

Совмин Белоруссии надеется разгрузить склады в Сибири
ORG─── LOC───────                              LOC───
Скопившаяся на складах белорусских предприятий продукция может быть 
реализована в Сибири при помощи административного ресурса российских 
              LOC───                                                 
чиновников. Об этом 17 июня заявил председатель правительства 
Белоруссии Михаил Мясникович на встрече с делегацией Сибирского 
LOC─────── PER──────────────                         LOC────────
федерального округа России во главе с полномочным представителем 
─────────────────── LOC───                                       
президента РФ Виктором Толоконским.
           LO PER───────────────── 
Во время встречи Мясникович указал на "огромный потенциал" 
                 PER───────                                
сотрудничества Белоруссии с Сибирским федеральным округом: по итогам 
               LOC───────   LOC──────────────────────────            
2012 года взаимный товароо

In [73]:
fileDir = r"collection3"
fileExt = r".ann"
documents_ann = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
print(documents_ann)

['356.ann', '149.ann', '368.ann', '183.ann', '252.ann', '094.ann', '100.ann', '537.ann', '409.ann', '130.ann', '623.ann', 'last_21.ann', '428.ann', '251.ann', '434.ann', '375.ann', '483.ann', '328.ann', '171.ann', '357.ann', 'artjakov.ann', '091.ann', '445.ann', '514.ann', '14_01_13i.ann', '279.ann', '058.ann', '374.ann', '299.ann', '042.ann', '243.ann', '263.ann', '431.ann', '310.ann', '362.ann', '25_12_12a.ann', '032.ann', '316.ann', '399.ann', '288.ann', '324.ann', '063.ann', '493.ann', '270.ann', '439.ann', '22_11_12d.ann', '192.ann', '170.ann', '118.ann', '446.ann', '506.ann', '309.ann', '23_11_12b.ann', '191.ann', '072.ann', '045.ann', '449.ann', '575.ann', '109.ann', '093.ann', '21_11_12c.ann', '20_11_12i.ann', '244.ann', '410.ann', '412.ann', '326.ann', '461.ann', '014.ann', '343.ann', '10_01_13a.ann', '146.ann', '377.ann', '038.ann', '09_01_13c.ann', '532.ann', '281.ann', '119.ann', '061.ann', '067.ann', '029.ann', '459.ann', 'chaves.ann', '286.ann', '289.ann', '240.ann', '233

In [74]:
ann = pd.read_csv('collection3/001.ann', delimiter='\t', header=None)
ann

,0,1,2
0,T1,LOC 0 6,Россия
1,T2,LOC 50 53,США
2,T3,LOC 57 63,Грузию
3,T4,LOC 87 93,МОСКВА
4,T5,ORG 103 114,РИА Новости
5,T6,LOC 116 122,Россия
6,T7,LOC 141 144,США
7,T8,LOC 161 168,Тбилиси
8,T9,LOC 301 307,России
9,T10,PER 308 324,Григорий Карасин


In [98]:
docs = []
# print(len(data_text))
# print(len(documents_ann))
# for i in range(len(data_text)):
for i in range(len(documents_ann)):
    words = []
    labels = []
    text = data_text['text'][i]
    # print(documents_ann[i])
    df = pd.read_csv('Collection3/' + documents_ann[i], delimiter='\t', header=None, quoting=csv.QUOTE_NONE, encoding='utf-8')
    df_ann = pd.DataFrame()
    df_ann['Token'] = df.loc[:, 2]
    split_1 = [loc.split() for loc in df.loc[:, 1].values]
    df_ann['Entity'] = [loc[0] for loc in split_1]
       
    dic = {}
    for j in range(len(df)):
        token = df_ann['Token'][j].lower().split()
        entity = df_ann['Entity'][j]
        for tok in token:
            dic[tok] = entity

    for token in tokenize(text):
        if (token.text.lower() in dic.keys()):
            words.append(token.text)
            labels.append(dic[token.text.lower()])
        else:
            words.append(token.text)
            labels.append('OUT')
    
    docs.append([words, labels])

In [100]:
print(docs[0][0]), print(docs[0][1])

['Совмин', 'Белоруссии', 'надеется', 'разгрузить', 'склады', 'в', 'Сибири', 'Скопившаяся', 'на', 'складах', 'белорусских', 'предприятий', 'продукция', 'может', 'быть', 'реализована', 'в', 'Сибири', 'при', 'помощи', 'административного', 'ресурса', 'российских', 'чиновников', '.', 'Об', 'этом', '17', 'июня', 'заявил', 'председатель', 'правительства', 'Белоруссии', 'Михаил', 'Мясникович', 'на', 'встрече', 'с', 'делегацией', 'Сибирского', 'федерального', 'округа', 'России', 'во', 'главе', 'с', 'полномочным', 'представителем', 'президента', 'РФ', 'Виктором', 'Толоконским', '.', 'Во', 'время', 'встречи', 'Мясникович', 'указал', 'на', '"', 'огромный', 'потенциал', '"', 'сотрудничества', 'Белоруссии', 'с', 'Сибирским', 'федеральным', 'округом', ':', 'по', 'итогам', '2012', 'года', 'взаимный', 'товарооборот', 'составил', 'более', '$', '1,4', 'млрд', '-', 'на', '4,4', '%', 'больше', 'уровня', '2011', 'года', '.', 'Председатель', 'Совмина', 'Белоруссии', 'с', 'сожалением', 'отметил', ':', 'в', 'I

(None, None)

In [101]:
data, labels = list(zip(*docs))
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')

Совмин	OUT
Белоруссии	OUT
надеется	OUT
разгрузить	OUT
склады	OUT
в	OUT
Сибири	OUT
Скопившаяся	OUT
на	OUT
складах	OUT
белорусских	OUT
предприятий	OUT
продукция	OUT
может	OUT
быть	OUT
реализована	OUT
в	OUT
Сибири	OUT
при	OUT
помощи	OUT
административного	OUT
ресурса	OUT
российских	OUT
чиновников	OUT
.	OUT
Об	OUT
этом	OUT
17	OUT
июня	OUT
заявил	OUT
председатель	OUT
правительства	OUT
Белоруссии	OUT
Михаил	OUT
Мясникович	OUT
на	OUT
встрече	OUT
с	OUT
делегацией	OUT
Сибирского	OUT
федерального	OUT
округа	OUT
России	OUT
во	OUT
главе	OUT
с	OUT
полномочным	OUT
представителем	OUT
президента	OUT
РФ	OUT
Виктором	OUT
Толоконским	OUT
.	OUT
Во	OUT
время	OUT
встречи	OUT
Мясникович	OUT
указал	OUT
на	OUT
"	OUT
огромный	OUT
потенциал	OUT
"	OUT
сотрудничества	OUT
Белоруссии	OUT
с	OUT
Сибирским	OUT
федеральным	OUT
округом	OUT
:	OUT
по	OUT
итогам	OUT
2012	OUT
года	OUT
взаимный	OUT
товарооборот	OUT
составил	OUT
более	OUT
$	OUT
1,4	OUT
млрд	OUT
-	OUT
на	OUT
4,4	OUT
%	OUT
больше	OUT
уровня	OUT
2011	OUT
года	OUT


In [102]:
df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j],
                   'data': [i for j in data for i in j],
                   'entities': [i for j in labels for i in j]})
df.head(50)

,sent_id,data,entities
0,0,Совмин,OUT
1,0,Белоруссии,OUT
2,0,надеется,OUT
3,0,разгрузить,OUT
4,0,склады,OUT
5,0,в,OUT
6,0,Сибири,OUT
7,0,Скопившаяся,OUT
8,0,на,OUT
9,0,складах,OUT


In [103]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = SentenceGetter(df)

sentences = getter.sentences

In [104]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i - 1][0]
        features.update({
            '-1:word.lower()': word1.lower()
        })
    else:
        features['BOS'] = True
    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        features.update({
            '+1:word.lower()': word1.lower()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [105]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [106]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X[2][7]

{'bias': 1.0,
 'word.lower()': 'на',
 'word[-3:]': 'на',
 'word[-2:]': 'на',
 'word.isdigit()': False,
 '-1:word.lower()': 'еще',
 '+1:word.lower()': '15'}

In [107]:
len(X)

713

In [108]:
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [115]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', # Градиентный спуск с использованием метода L-BFGS
    c1=0.1, # Коэффициент для регуляризации L1
    c2=0.1, # Коэффициент для регуляризации L2
    max_iterations=1000, # Максимальное количество итераций
    all_possible_transitions=True, # Генерация объектов (не встречающихся в обучающих данных)
    verbose=False # Выключение режима тренировки
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

CPU times: user 28 s, sys: 89 ms, total: 28.1 s
Wall time: 29.3 s


In [116]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [117]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.9853420963702066

In [118]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [119]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X[1][7]

{'bias': 1.0,
 'word.lower()': 'путина',
 'word[-3:]': 'ина',
 'word[-2:]': 'на',
 'word.isdigit()': False}

In [120]:
len(X)

713

In [121]:
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [123]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True,
    verbose=False
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

CPU times: user 32.9 s, sys: 101 ms, total: 33 s
Wall time: 33.3 s


In [124]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [125]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.9855229392577891